In [21]:
# from Database import Database
# import sys
# # import multiprocessing as mp
# import pandas as pd
# from collections import Counter
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from scipy.stats.distributions import chi2
# from bioinfokit import analys, visuz
# from scipy.stats import fisher_exact
# import time
# from scipy.special import factorial


# from fisher import pvalue_npy
# from scipy.stats import chi2_contingency
# from scipy.stats import uniform, randint

from Database import Database
import pandas as pd
from collections import Counter
# Python program to create
# sparse matrix using csr_matrix()
# Import required package
import numpy as np
from scipy.sparse import csr_matrix
# from matplotlib import pyplot as plt

In [22]:
path_db = "D:/Hanze_Groningen/STAGE/db_laatste_copy.db" #"/groups/umcg-wijmenga/tmp01/projects/lude_vici_2021/rawdata/cancer_data/new_db/copydb_L.db"  # /groups/umcg-wijmenga/tmp01/projects/lude_vici_2021/rawdata/cancer_data/new_db/copydatabase_C.db
# Database connection
db = Database(path_db)

df = pd.read_sql('''SELECT project.cancer, sum_dosage_GT.donor_project_ID, 
                        sum_dosage_GT.donor_ID, sum_dosage_GT.snp_ID, 
                        snp.chr, snp.pos_start, snp.pos_end , snp.DNase, snp.TFBS, snp.UCNE
                FROM project, sum_dosage_GT, snp 
                WHERE sum_dosage_GT.snp_ID=snp.ID AND 
                            sum_dosage_GT.donor_project_ID = project.ID AND 
                            (sum_dosage_GT.GT2 = 1 OR sum_dosage_GT.GT2 = 2) AND 
                            sum_dosage_GT.total_read_count_sum >= 33 ;''', db.mydb_connection)

In [ ]:
def layer_run(df_variant, name_variant):    
    header_file = 'filter\tgene\tchr\tstart_position_regio\tend_position_regio\t#snp_unique\tsnp_list\t#donors_all' \
                '\tdonor_count\tcancer_count\n'    
    # Make file
    f = open(f'D:/Hanze_Groningen/STAGE/{name_variant}_num_snps_ALL.tsv', 'w')
    f.write(header_file)
    for index, row in df_variant.iterrows():
        print(index)
        x = df[(df['chr']==row['#Chromosome']) & (df['pos_start']>=row['Start']) & (df['pos_end']<=row['End'])]
        donor_list = list(x['donor_ID'])
        donor_count = dict(Counter(donor_list))
        cancer_list = list(x['cancer'])
        cancer_count = dict(Counter(cancer_list))
        if len(x) > 0:
            f.write(str(33) + '\t' + str(row['#Chromosome']) + '\t' + str(row['Start']) + '\t' + str(row['End']) + '\t' + str(
            len(x['snp_ID'])) + '\t' + ','.join(map(str, list(x['snp_ID']))) + '\t' + str(len(donor_list)) + '\t' + str(
            donor_count) + '\t' + str(cancer_count) + '\n')
        else:
            f.write(str(33) + '\t' + str(row['#Chromosome']) + '\t' + str(row['Start']) + '\t' + str(row['End']) + '\t-\t-\t-\t-\t-\n')

    f.close()


In [ ]:
# DNase
path_file = 'D:/Hanze_Groningen/STAGE/lagen/2022-04-13_GRCh37_DNase.merged.bed.gz'
df_variant = pd.read_csv(path_file, sep='\t', compression='gzip')
df_variant['#Chromosome'] = df_variant['#Chromosome'].str.replace('chr', '')
print(len(df_variant))

In [ ]:
name_variant = 'DNase'
layer_run(df_variant, name_variant)

In [ ]:
# TFBS
path_file = 'D:/Hanze_Groningen/STAGE/lagen/2022-04-13_GRCh37_TFBS.merged.bed.gz'
df_variant = pd.read_csv(path_file, sep='\t', compression='gzip')
df_variant['#Chromosome'] = df_variant['#Chromosome'].str.replace('chr', '')
print(len(df_variant))


In [ ]:
name_variant = 'TFBS'
layer_run(df_variant, name_variant)

In [ ]:
path_db = 'D:/Hanze_Groningen/STAGE/lastdb/db_laatste_copy.db' #'D:/Hanze_Groningen/STAGE/DATAB/copydatabase_C.db'
# Database connection
db = Database(path_db)

In [ ]:
table = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", db.mydb_connection)
table


In [ ]:
results = db.cursor.execute("PRAGMA table_info(project)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(sqlite_sequence)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(donor)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(snp)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(tissue)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(donor_has_snp)").fetchall()
for res in results:
    print(res[:])

In [ ]:
results = db.cursor.execute("PRAGMA table_info(sum_dosage_GT)").fetchall()
for res in results:
    print(res[:])

In [ ]:
for res in results:
    print(res[0], res[1])
    donor = db.cursor.execute(
                """SELECT donor_ID
                    FROM donor
                    WHERE project_ID = %s ;""" %
                int(res[0])).fetchall()
    set_don = list()
    for don in donor:
        set_don.append(don[0])
    print(len(set(set_don)))
print(len(results))

In [ ]:
results = db.cursor.execute("""SELECT cancer, ID
                            FROM project""").fetchall()
dict_cancer = dict()
for res in results:
    if res[0] in dict_cancer:
        dict_cancer[res[0]].append(res[1])
    else:
        dict_cancer[res[0]] = [res[1]]

dict_2 = dict()
for key, value in dict_cancer.items():
    donor_num = 0
    for ID in value:
        donor = db.cursor.execute(
                    """SELECT donor_ID
                        FROM donor
                        WHERE project_ID = %s ;""" %
                    int(ID)).fetchall()
        set_don = list()
        for don in donor:
            set_don.append(don[0])
        donor_num += len(set(set_don))
    dict_2[key] = donor_num

print(dict_2)